## Project 1: Class Scheduling with Mathematical Programming

* **Collaborators**: Isidore Sossa and Chase Scott
* **Resources**: 
    * Nurses Scheduling Problem
* **Project's requirements**:
    * Use data provided through Canvas
    * Generate random list of preferences for each professor according to their qualification, i.e. an integer from 1 to *n*, where *n* is the number of classes the professor is qualified to teach. The higher the number the lesser the preference.
    * Extract out professors who do not teach
    * Professors who do teach work at the maximum of their workload
    * Classes start at 8:00 AM and end at 6:00 PM and can only start at every half period.
    * One credit hour is equivalent to 50 minutes. E.g. A 3-credit hour class meet for a total of 150 minutes per week.
    * Add two constraints of your chosen

In [1]:
import pandas as pd
from docplex.mp.model import Model
import math
import random
import numpy as np
import itertools

### Functions and Customizations

In [2]:
## Standardize data frame column names
def clean_dataframe_columns(df:pd.DataFrame) -> pd.DataFrame:
    colnames = df.columns.values.tolist()
    colnames = [col.replace(' ', '_') for col in colnames]
    df.columns = colnames

In [3]:
## Duplicate courses with multiple sessions
def standardize_courses_offered(courses_offered:pd.DataFrame, course_catalog_df:pd.DataFrame) -> pd.DataFrame:
    ''''''
    def get_day_name(day:str):
        days_name = {'M':'Monday', 'TU':'Tuesday', 'W':'Wednesday', 'TH':'Thursday', 'F':'Friday'}
        return days_name[day]
    
    result = pd.DataFrame()
    result = result.append(courses_offered[courses_offered['Sections'] == 1])
    for course in courses_offered[courses_offered['Sections'] > 1].itertuples():
        course = pd.DataFrame(course).T.drop(columns=0)
        course.columns = courses_offered.columns
        for i in range(1, course['Sections'][0] + 1):
            course['Sections'] = i
            result = result.append([course], ignore_index=True)
    result = result.rename(columns={'Sections' : 'Session'})
    result = result.sort_values(by=['Course_Number', 'Session']).reset_index().drop(columns='index')
    result = pd.merge(left=result, right=course_catalog_df, on=['Department_Code', 'Course_Number', 'Course_Name'])
    
    courses_offered = result
    result = pd.DataFrame()
    
    for course in courses_offered.itertuples():
        course = pd.DataFrame(course).T.drop(columns=0)
        course.columns = courses_offered.columns
        days_taught = course['Day'][0]
        days_taught = days_taught.split('/')
        for i in days_taught:
            course['Day'] = i
            result = result.append([course], ignore_index=True)
            
    result['Day'] = result.apply(lambda row : get_day_name(row.Day), axis=1)
    return result

In [4]:
## Clean
def merge_and_get_preference(professors_df:pd.DataFrame, qualifications:pd.DataFrame, course:str) -> pd.DataFrame:
    ''''''
    def manipulate_row(row:pd.DataFrame, course:str):
        cols = [x for x in row.columns if course in x]
        class_taught = 0
        for col in cols:
            if not math.isnan(row[col][0]):
                class_taught += row[col][0]
        class_taught = int(class_taught)
        if row['Workload_Credit_Hours'][0] > 0:
            preferences = random.sample(list(range(1, class_taught + 1)), class_taught)
        else:
            preferences = [0] * class_taught
            
        #display(preferences)

        index = 0
        for col in cols:
            if not math.isnan(row[col][0]):
                row[col] = preferences[index]
                index += 1
            else:
                row[col] = 0
        return row

    result_df = pd.merge(left=professors_df, right=qualifications_df, on=['Faculty_Name', 'Department_Code'])
    col_names = list(result_df.columns)
    preferences_df = pd.DataFrame(columns=col_names)
    col_names = result_df.columns
    for row in result_df.itertuples():
        row_df = pd.Series(row).to_frame().T.drop(columns=0)
        row_df.columns = col_names
        row_df = manipulate_row(row_df, course)
        preferences_df = preferences_df.append(row_df, ignore_index=True)
    return preferences_df

In [5]:
def get_period_df(start_hour=8, end_hour=18) -> pd.DataFrame:
    '''Create a dataframe with days'''
    days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']
    days_of_weeks = dict(zip(days, range(0, 5)))
    days_of_weeks

    result = pd.DataFrame(columns=['Period', 'Day'])
    start = 1
    day_digits = []
    day_names = []
    for day, day_digit in days_of_weeks.items():
        day_names.extend([day] * 20)
        day_digits.extend([day_digit] * 20)
        
    start_time = [x * 0.5 for x in range(2*start_hour, 2*end_hour)] * len(days)
    end_time = [x * 0.5 for x in range(2*start_hour + 1, 2*end_hour + 1)] * len(days)
    
    
    return pd.DataFrame.from_dict({'Period' : list(range(1, 101)), 'Day_Name' : day_names, 'Day_Number' : day_digits, 
                                   'Start_Time' : start_time, 'End_Time' : end_time})


In [6]:
def get_column_value_as_list(df:pd.DataFrame, colname:str) -> pd.DataFrame:
    '''Get the values of a data frame column as a Python list'''
    if colname in df.columns:
        return df[colname].values.tolist()
    else:
        return None

### Data Exploration and Cleaning

In [7]:
## Load data
data_file = 'Data Set – Class Schedule.xlsx'
file_handler = pd.ExcelFile(path_or_buffer=data_file)
file_sheets = sorted(file_handler.sheet_names)
print(f'Sheets : {file_sheets}')

## Load each sheet as a DataFrame
classrooms_df = pd.read_excel(io=file_handler, sheet_name=file_sheets[0])
clean_dataframe_columns(classrooms_df)
display(file_sheets[0], classrooms_df)

colleges_and_depts = pd.read_excel(io=file_handler, sheet_name=file_sheets[1])
clean_dataframe_columns(colleges_and_depts)
display(file_sheets[1], colleges_and_depts.head(5))

course_catalog_df = pd.read_excel(io=file_handler, sheet_name=file_sheets[2])
clean_dataframe_columns(course_catalog_df)
display(file_sheets[2], course_catalog_df.sort_values('Course_Name').head(25))

courses_offered = pd.read_excel(io=file_handler, sheet_name=file_sheets[3])
clean_dataframe_columns(courses_offered)
display(file_sheets[3], courses_offered.sort_values('Course_Name').head(45))

professors_df = pd.read_excel(io=file_handler, sheet_name=file_sheets[4])
clean_dataframe_columns(professors_df)
display(file_sheets[4], professors_df.head(5))

qualifications_df = pd.read_excel(io=file_handler, sheet_name=file_sheets[5])
clean_dataframe_columns(qualifications_df)
display(file_sheets[5], qualifications_df.head(5))

Sheets : ['Classrooms', 'Colleges and Departments', 'Course Catalog', 'Courses Offered Fall 2021', 'Professors', 'Qualification']


'Classrooms'

,Building,Building_Code,Classroom,Capacity
0,Durland,DU,1029,47
1,Durland,DU,1041,24
2,Durland,DU,1052,60
3,Durland,DU,1061,47
4,Durland,DU,1063,47
5,Durland,DU,1066,91
6,Durland,DU,1069,22
7,Durland,DU,1073,185


'Colleges and Departments'

,College,College_Code,Department,Department_Code
0,College of Agriculture,CAGRI,Entomology,ENTOM


...

,Department_Code,Course_Number,Course_Name,Sections,Day,Max_Enrollement
25,ENTOM,835,Advanced Insect Evolution,1,M/W/F,20
23,ENTOM,825,Advanced Integrative Behavioral Ecology,1,M/W,20
2,ENTOM,305,Animal Health Entomology,3,TU/TH,50
3,ENTOM,306,Animal Health Entomology Laboratory,1,M/W/F,50
15,ENTOM,680,Aquatic Entomology,1,M/W/F,40
28,ENTOM,849,Arthropod Vectors of Human Pathogens,1,M/W/F,20
33,ENTOM,885,Conventional and Molecular Methods for Evaluat...,1,M/W,20
4,ENTOM,350,"Crops, Insects, and Agroecosystems",1,M/W,50
32,ENTOM,880,Ecological Genomics,1,M/TU/W/TH,20
0,ENTOM,300,Economic Entomology,1,M/W/F,50


'Professors'

,Department_Code,Faculty_Name,Workload_Credit_Hours
0,ENTOM,Frank H. Arthur,6
1,ENTOM,James F. Campbell,9
2,ENTOM,Ming-Shun Chen,9
3,ENTOM,Raymond Cloyd,9
4,ENTOM,Lee Cohnstaedt,9


'Qualification'

,Department_Code,Faculty_Name,ENTOM_300,ENTOM_301,ENTOM_305,ENTOM_306,ENTOM_350,ENTOM_589,ENTOM_602,ENTOM_621,...,ENTOM_830,ENTOM_835,ENTOM_837,ENTOM_840,ENTOM_849,ENTOM_857,ENTOM_860,ENTOM_875,ENTOM_880,ENTOM_885
0,ENTOM,Frank H. Arthur,NaN,1.0,1.0,NaN,1.0,NaN,NaN,NaN,...,1.0,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN
1,ENTOM,James F. Campbell,NaN,1.0,1.0,NaN,1.0,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,1.0,1.0
2,ENTOM,Ming-Shun Chen,NaN,1.0,1.0,1.0,NaN,1.0,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0
3,ENTOM,Raymond Cloyd,NaN,1.0,1.0,1.0,1.0,1.0,NaN,NaN,...,NaN,1.0,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN
4,ENTOM,Lee Cohnstaedt,1.0,1.0,1.0,1.0,1.0,NaN,1.0,NaN,...,1.0,NaN,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN


In [8]:
# For reproducibility
random.seed(1345)

courses_offered = standardize_courses_offered(courses_offered, course_catalog_df)
preferences_df = merge_and_get_preference(professors_df, qualifications_df, 'ENTOM')
display(preferences_df)
periods_df = get_period_df()
display(periods_df.head(21))

,Department_Code,Faculty_Name,Workload_Credit_Hours,ENTOM_300,ENTOM_301,ENTOM_305,ENTOM_306,ENTOM_350,ENTOM_589,ENTOM_602,...,ENTOM_830,ENTOM_835,ENTOM_837,ENTOM_840,ENTOM_849,ENTOM_857,ENTOM_860,ENTOM_875,ENTOM_880,ENTOM_885
0,ENTOM,Frank H. Arthur,6,0,2,8,0,5,0,0,...,3,0,0,4,1,0,0,0,0,0
1,ENTOM,James F. Campbell,9,0,6,8,0,4,1,0,...,0,0,0,0,3,0,0,0,7,5
2,ENTOM,Ming-Shun Chen,9,0,6,2,3,0,8,0,...,0,0,0,0,0,0,0,1,7,5
3,ENTOM,Raymond Cloyd,9,0,4,11,3,2,8,0,...,0,9,0,0,10,7,0,0,0,0
4,ENTOM,Lee Cohnstaedt,9,6,3,4,5,10,0,2,...,11,0,0,0,7,0,9,0,0,0
5,ENTOM,Srinivas Kambhampati,6,0,0,6,3,0,0,0,...,0,0,0,9,0,0,7,0,8,0
6,ENTOM,Tania Kim,3,7,0,3,2,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,ENTOM,Berlin Luxelly Londono Renteria,6,4,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
8,ENTOM,Jeremy L. Marshall,9,7,3,2,6,0,0,8,...,0,0,0,0,0,0,0,0,0,0
9,ENTOM,Brian P. McCornack,9,0,0,0,1,0,0,0,...,3,2,0,5,7,0,0,0,0,0


,Period,Day_Name,Day_Number,Start_Time,End_Time
0,1,Monday,0,8.0,8.5
1,2,Monday,0,8.5,9.0
2,3,Monday,0,9.0,9.5
3,4,Monday,0,9.5,10.0
4,5,Monday,0,10.0,10.5
5,6,Monday,0,10.5,11.0
6,7,Monday,0,11.0,11.5
7,8,Monday,0,11.5,12.0
8,9,Monday,0,12.0,12.5
9,10,Monday,0,12.5,13.0


### Mathematical Model

#### Model Initialization

In [9]:
## Model Environment
model = Model(name='Course Scheduling')

######################
# Decision Variables #
######################
x_fcsrp = dict()
for professor in preferences_df.Faculty_Name:
    for course in courses_offered.itertuples():
        for period in periods_df[periods_df.Day_Name == course.Day].Period.values.tolist():
            for room in classrooms_df.Classroom:
                x_fcsrp[(professor, course.Course_Name, course.Session, room, period)] = model.binary_var(name = "x({},{},{},{},{})".format(professor, course.Course_Name, course.Session, room, period))
                
model.print_information()

Model: Course Scheduling
 - number of variables: 397440
   - binary=397440, integer=0, continuous=0
 - number of constraints: 0
   - linear=0
 - parameters: defaults
 - objective: none
 - problem type is: MILP


#### Constraints

In [10]:
### Exclude faculty member whose workload is zero
for professor in preferences_df[preferences_df.Workload_Credit_Hours == 0].itertuples():
    for course in courses_offered.itertuples():
        for period in periods_df[periods_df.Day_Name == course.Day].Period.values.tolist():
            for room in classrooms_df.Classroom:
                model.add_constraint(x_fcsrp[(professor.Faculty_Name, course.Course_Name, course.Session, room, period)] == 0)

                    
### Each course is taught for the number of periods needed to satisfy the course credit hours on the days it is offered
CLASS_DURATION = 50
PERIOD_DURATION = 30
for course in courses_offered.groupby(by=['Course_Name', 'Session']):
    course_name, session = course[0]
    number_of_credit = course[1].Credit_Hours.values.tolist()[0]
    duration = (number_of_credit * CLASS_DURATION) / (len(course[1]) * PERIOD_DURATION)
    number_of_period_covered = math.ceil(duration)
    for day_offered in course[1].Day.values.tolist():
        aggregate_values = []
        for period in periods_df[periods_df.Day_Name == day_offered].Period.values.tolist():
            for professor in preferences_df.Faculty_Name:
                for room in classrooms_df.Classroom:
                    aggregate_values.append(x_fcsrp[(professor, course_name, session, room, period)])
        model.add_constraint(model.sum(aggregate_values) == number_of_period_covered)
        

### A faculty can only teach one course at a time
for professor in preferences_df.Faculty_Name:
    for period in periods_df.Period:
        aggregate_values = []
        for course in courses_offered.itertuples():
            days_offered = periods_df[periods_df.Day_Name == course.Day].Period.values.tolist()
            if period in days_offered:
                for room in classrooms_df.Classroom:
                    aggregate_values.append(x_fcsrp[(professor, course.Course_Name, course.Session, room, period)])
        model.add_constraint(model.sum(aggregate_values) <= 1)        
        
### Each course is taught at the same time in the same room by the same faculty on the day it is offered
for course in courses_offered.groupby(by=['Course_Name', 'Session']):
    course_name, session = course[0]
    days_offered = course[1].Day.values.tolist()
    for index in range(len(days_offered) - 1):
        x = periods_df[periods_df.Day_Name == days_offered[index]].Period.values.tolist()
        y = periods_df[periods_df.Day_Name == days_offered[index + 1]].Period.values.tolist()
        for periods in list(zip(x, y)):
            for professor in preferences_df.Faculty_Name:
                for room in classrooms_df['Classroom']:
                    aggregate_values = []
                    aggregate_values.append(x_fcsrp[(professor, course_name, session, room, periods[0])])
                    aggregate_values.append(x_fcsrp[(professor, course_name, session, room, periods[1])])
                    model.add_constraint(aggregate_values[0] - aggregate_values[1] == 0)

            
### A course is taught in a classroom that can hold at least the maximum number of students enrolled
for course in courses_offered.itertuples():
    for period in periods_df[periods_df.Day_Name == course.Day].Period.values.tolist():
        for professor in preferences_df.Faculty_Name:
            for room in classrooms_df.itertuples():
                model.add_constraint(x_fcsrp[(professor, course.Course_Name, course.Session, room.Classroom, period)] * course.Max_Enrollement <= room.Capacity)


### Each faculty member can only teach up to his or her workload
CLASS_DURATION = 50
PERIOD_DURATION = 30
for professor in preferences_df.itertuples():
    total_hours_taught = 0
    for course in courses_offered.groupby(by=['Course_Name', 'Session']):
        aggregate_values = []
        course_name, session = course[0]
        number_of_credit = course[1].Credit_Hours.values.tolist()[0]
        number_of_meetings = len(course[1])
        duration = (number_of_credit * CLASS_DURATION) / (number_of_meetings * PERIOD_DURATION)
        number_of_period_covered = math.ceil(duration)
        for day_offered in course[1].Day.values.tolist():
            for period in periods_df[periods_df.Day_Name == day_offered].Period.values.tolist():
                for room in classrooms_df.Classroom:
                    aggregate_values.append(x_fcsrp[(professor.Faculty_Name, course_name, session, room, period)])
        total_hours_taught += sum(aggregate_values) * number_of_credit / (number_of_meetings * number_of_period_covered)
    model.add_constraint(total_hours_taught <= professor.Workload_Credit_Hours)

#### Special Rules

1. Seniors courses start at 1:00 PM

In [11]:
### 800+ courses are taught after 1:00 PM
SENIOR_COURSE_NUMBER = 800
START_TIME = 13 # PM
for course in courses_offered[courses_offered.Course_Number >= SENIOR_COURSE_NUMBER].itertuples():
    for period in periods_df[(periods_df.Day_Name == course.Day) & (periods_df.Start_Time < START_TIME)].Period.values.tolist():
        for professor in preferences_df.Faculty_Name:
            for room in classrooms_df.Classroom:
                model.add_constraint(x_fcsrp[(professor, course.Course_Name, course.Session, room, period)] == 0)


2. No classes are taught on Friday after 4 PM

In [12]:
HOLIDAY = 'Friday'
END_TIME = 16
for course in courses_offered[courses_offered.Day == HOLIDAY].itertuples():
    for period in periods_df[(periods_df.Day_Name == HOLIDAY) & (periods_df.Start_Time >= END_TIME)].Period.values.tolist():
        for professor in preferences_df.Faculty_Name:
            for room in classrooms_df.Classroom:
                model.add_constraint(x_fcsrp[(professor, course.Course_Name, course.Session, room, period)] == 0)

#### Objective Function

In [13]:
total_preference = 0
for professor in preferences_df.itertuples():
    professor_df = pd.DataFrame(professor).T.drop(columns=0)
    professor_df.columns = preferences_df.columns
    for course in courses_offered.itertuples():
        course_code = '{}_{}'.format(course.Department_Code, str(course.Course_Number))
        preference_score = professor_df[course_code].values.tolist()[0]
        professor_name = professor_df.Faculty_Name.values.tolist()[0]
        for period in periods_df[periods_df.Day_Name == course.Day].Period.values.tolist():
            for room in classrooms_df.Classroom:
                total_preference += preference_score * x_fcsrp[(professor_name, course.Course_Name, course.Session, room, period)]

model.maximize(total_preference)
model.print_information()

Model: Course Scheduling
 - number of variables: 397440
   - binary=397440, integer=0, continuous=0
 - number of constraints: 822979
   - linear=822979
 - parameters: defaults
 - objective: maximize
 - problem type is: MILP


#### Solution

In [27]:
###############
# Solve Model #
###############
model.print_information()
solution = model.solve(log_output = True)
assert solution, "Solve Failed"
model.report()

Model: Course Scheduling
 - number of variables: 397440
   - binary=397440, integer=0, continuous=0
 - number of constraints: 822979
   - linear=822979
 - parameters: defaults
 - objective: maximize
 - problem type is: MILP
Version identifier: 20.1.0.0 | 2020-11-10 | 9bedb6d68
CPXPARAM_Read_DataCheck                          1

Root node processing (before b&c):
  Real time             =    0.03 sec. (10.23 ticks)
Parallel b&c, 8 threads:
  Real time             =    0.00 sec. (0.00 ticks)
  Sync time (average)   =    0.00 sec.
  Wait time (average)   =    0.00 sec.
                          ------------
Total (root+branch&cut) =    0.03 sec. (10.23 ticks)
* model Course Scheduling solved with objective = 1889.000


#### Post-processing

In [23]:
########################
## View model solution #
########################
classes_assigned = []
professors_assigned = []
professors = []
courses = []
course_codes = []
rooms = []
sessions = []
periods = []
days = []
workloads = []
credits = []
room_capacity = []
course_enrollements = []
#print(f"{'Professor':<35} {'Course Name':<40} {'Session':>10} {'Room':>5} {'Period':>10} {'Day Offered':<10}")
for course in courses_offered.groupby(by=['Course_Name', 'Session']):
    course_name, session = course[0]
    course_code = course[1].Course_Number.values.tolist()[0]
    course_credit = course[1].Credit_Hours.values.tolist()[0]
    course_enrollment = course[1].Max_Enrollement.values.tolist()[0]
    for day_offered in course[1].Day.values.tolist():
        for professor in preferences_df.itertuples():
            for period in periods_df[periods_df.Day_Name == day_offered].Period.values.tolist():
                for room in classrooms_df.itertuples():
                    value = model.solution.get_value(x_fcsrp[(professor.Faculty_Name, course_name, session, room.Classroom, period)])
                    if value != 0:
                        professors.append(professor.Faculty_Name)
                        courses.append(course_name)
                        course_codes.append(course_code)
                        sessions.append(session)
                        rooms.append(room.Classroom)
                        days.append(day_offered)
                        periods.append(period)
                        workloads.append(professor.Workload_Credit_Hours)
                        credits.append(course_credit)
                        room_capacity.append(room.Capacity)
                        course_enrollements.append(course_enrollment)
                        
                        #print(f'{professor:<35} {course_name:40} {session:>10} {room:>5} {period:>10} {day_offered:<10}')
                        if course_name not in classes_assigned:
                            classes_assigned.append(course_name)
                        if professor_name not in professors_assigned:
                            professors_assigned.append(professor_name)
    #print('\n\n')
    
print(f'Number of courses assigned {len(classes_assigned)}')
print(f'Number of courses offered  {len(courses_offered.groupby(by=["Course_Name", "Session"]))}')
print("\nAll of the following courses have been assigned")
print(set(classes_assigned).difference(set(courses_offered.Course_Name.values.tolist())))
print(set(courses_offered.Course_Name.values.tolist()).difference(set(classes_assigned)))

print(set(professors_assigned).difference(set(qualifications_df.Faculty_Name.values.tolist())))

class_schedule = pd.DataFrame({'Professor' : professors, 'Workload' : workloads, 'Course_Name': courses, 'Course_Code' : course_codes, 'Credit' : credits, 'Session' : sessions, 'Period' : periods, 'Day_Offered' : days, 'Max_Enrollement' : course_enrollements, 'Classroom' : rooms, 'Room Capacity' : room_capacity})
#display(class_schedule)

class_schedule.sort_values(by=['Course_Name', 'Session', 'Day_Offered', 'Period'])
for course in class_schedule.groupby(by='Professor'):
    display(course[1])

Number of courses assigned 34
Number of courses offered  38

All of the following courses have been assigned
set()
set()
set()


,Professor,Workload,Course_Name,Course_Code,Credit,Session,Period,Day_Offered,Max_Enrollement,Classroom,Room Capacity
36,Brenda Oppert,3,Aquatic Entomology,680,3,1,1,Monday,40,1029,47
37,Brenda Oppert,3,Aquatic Entomology,680,3,1,2,Monday,40,1029,47
38,Brenda Oppert,3,Aquatic Entomology,680,3,1,41,Wednesday,40,1029,47
39,Brenda Oppert,3,Aquatic Entomology,680,3,1,42,Wednesday,40,1029,47
40,Brenda Oppert,3,Aquatic Entomology,680,3,1,81,Friday,40,1029,47
41,Brenda Oppert,3,Aquatic Entomology,680,3,1,82,Friday,40,1029,47


,Professor,Workload,Course_Name,Course_Code,Credit,Session,Period,Day_Offered,Max_Enrollement,Classroom,Room Capacity
42,Brian P. McCornack,9,Arthropod Vectors of Human Pathogens,849,3,1,11,Monday,20,1066,91
43,Brian P. McCornack,9,Arthropod Vectors of Human Pathogens,849,3,1,16,Monday,20,1029,47
44,Brian P. McCornack,9,Arthropod Vectors of Human Pathogens,849,3,1,51,Wednesday,20,1066,91
45,Brian P. McCornack,9,Arthropod Vectors of Human Pathogens,849,3,1,56,Wednesday,20,1029,47
46,Brian P. McCornack,9,Arthropod Vectors of Human Pathogens,849,3,1,91,Friday,20,1066,91
47,Brian P. McCornack,9,Arthropod Vectors of Human Pathogens,849,3,1,96,Friday,20,1029,47
96,Brian P. McCornack,9,Insect Evolution,635,3,1,21,Tuesday,40,1052,60
97,Brian P. McCornack,9,Insect Evolution,635,3,1,22,Tuesday,40,1061,47
98,Brian P. McCornack,9,Insect Evolution,635,3,1,23,Tuesday,40,1073,185
99,Brian P. McCornack,9,Insect Evolution,635,3,1,61,Thursday,40,1052,60


,Professor,Workload,Course_Name,Course_Code,Credit,Session,Period,Day_Offered,Max_Enrollement,Classroom,Room Capacity
6,Brian Spiesman,9,Advanced Integrative Behavioral Ecology,825,3,1,11,Monday,20,1029,47
7,Brian Spiesman,9,Advanced Integrative Behavioral Ecology,825,3,1,12,Monday,20,1063,47
8,Brian Spiesman,9,Advanced Integrative Behavioral Ecology,825,3,1,13,Monday,20,1029,47
9,Brian Spiesman,9,Advanced Integrative Behavioral Ecology,825,3,1,51,Wednesday,20,1029,47
10,Brian Spiesman,9,Advanced Integrative Behavioral Ecology,825,3,1,52,Wednesday,20,1063,47
11,Brian Spiesman,9,Advanced Integrative Behavioral Ecology,825,3,1,53,Wednesday,20,1029,47
121,Brian Spiesman,9,Insect Taxonomy,710,3,1,1,Monday,30,1029,47
122,Brian Spiesman,9,Insect Taxonomy,710,3,1,9,Monday,30,1029,47
124,Brian Spiesman,9,Insect Taxonomy,710,3,1,41,Wednesday,30,1029,47
125,Brian Spiesman,9,Insect Taxonomy,710,3,1,49,Wednesday,30,1029,47


,Professor,Workload,Course_Name,Course_Code,Credit,Session,Period,Day_Offered,Max_Enrollement,Classroom,Room Capacity
114,Erin Scully,3,Insect Physiology,875,3,1,36,Tuesday,20,1063,47
115,Erin Scully,3,Insect Physiology,875,3,1,37,Tuesday,20,1029,47
116,Erin Scully,3,Insect Physiology,875,3,1,38,Tuesday,20,1063,47
117,Erin Scully,3,Insect Physiology,875,3,1,76,Thursday,20,1063,47
118,Erin Scully,3,Insect Physiology,875,3,1,77,Thursday,20,1029,47
119,Erin Scully,3,Insect Physiology,875,3,1,78,Thursday,20,1063,47


...

,Professor,Workload,Course_Name,Course_Code,Credit,Session,Period,Day_Offered,Max_Enrollement,Classroom,Room Capacity
144,Raymond Cloyd,9,Insects of Stored Products,805,3,1,31,Tuesday,20,1029,47
145,Raymond Cloyd,9,Insects of Stored Products,805,3,1,32,Tuesday,20,1061,47
146,Raymond Cloyd,9,Insects of Stored Products,805,3,1,34,Tuesday,20,1061,47
147,Raymond Cloyd,9,Insects of Stored Products,805,3,1,71,Thursday,20,1029,47
148,Raymond Cloyd,9,Insects of Stored Products,805,3,1,72,Thursday,20,1061,47
149,Raymond Cloyd,9,Insects of Stored Products,805,3,1,74,Thursday,20,1061,47
202,Raymond Cloyd,9,Plant Resistance to Insects,655,3,1,21,Tuesday,40,1066,91
203,Raymond Cloyd,9,Plant Resistance to Insects,655,3,1,33,Tuesday,40,1063,47
205,Raymond Cloyd,9,Plant Resistance to Insects,655,3,1,61,Thursday,40,1066,91
206,Raymond Cloyd,9,Plant Resistance to Insects,655,3,1,73,Thursday,40,1063,47


,Professor,Workload,Course_Name,Course_Code,Credit,Session,Period,Day_Offered,Max_Enrollement,Classroom,Room Capacity
84,Srinivas Kambhampati,6,Immature Insects,840,3,1,11,Monday,20,1063,47
85,Srinivas Kambhampati,6,Immature Insects,840,3,1,12,Monday,20,1029,47
86,Srinivas Kambhampati,6,Immature Insects,840,3,1,51,Wednesday,20,1063,47
87,Srinivas Kambhampati,6,Immature Insects,840,3,1,52,Wednesday,20,1029,47
88,Srinivas Kambhampati,6,Immature Insects,840,3,1,91,Friday,20,1063,47
89,Srinivas Kambhampati,6,Immature Insects,840,3,1,92,Friday,20,1029,47
102,Srinivas Kambhampati,6,Insect Genetics,860,3,1,37,Tuesday,20,1063,47
103,Srinivas Kambhampati,6,Insect Genetics,860,3,1,38,Tuesday,20,1052,60
105,Srinivas Kambhampati,6,Insect Genetics,860,3,1,77,Thursday,20,1063,47
106,Srinivas Kambhampati,6,Insect Genetics,860,3,1,78,Thursday,20,1052,60


,Professor,Workload,Course_Name,Course_Code,Credit,Session,Period,Day_Offered,Max_Enrollement,Classroom,Room Capacity
214,Tania Kim,3,Problems in Entomology,799,3,1,11,Monday,30,1029,47
215,Tania Kim,3,Problems in Entomology,799,3,1,12,Monday,30,1029,47
216,Tania Kim,3,Problems in Entomology,799,3,1,51,Wednesday,30,1029,47
217,Tania Kim,3,Problems in Entomology,799,3,1,52,Wednesday,30,1029,47
218,Tania Kim,3,Problems in Entomology,799,3,1,91,Friday,30,1029,47
219,Tania Kim,3,Problems in Entomology,799,3,1,92,Friday,30,1029,47


,Professor,Workload,Course_Name,Course_Code,Credit,Session,Period,Day_Offered,Max_Enrollement,Classroom,Room Capacity
13,Weston Opitz,9,Animal Health Entomology,305,3,1,22,Tuesday,50,1052,60
14,Weston Opitz,9,Animal Health Entomology,305,3,1,27,Tuesday,50,1052,60
16,Weston Opitz,9,Animal Health Entomology,305,3,1,62,Thursday,50,1052,60
17,Weston Opitz,9,Animal Health Entomology,305,3,1,67,Thursday,50,1052,60
19,Weston Opitz,9,Animal Health Entomology,305,3,2,21,Tuesday,50,1066,91
20,Weston Opitz,9,Animal Health Entomology,305,3,2,29,Tuesday,50,1052,60
22,Weston Opitz,9,Animal Health Entomology,305,3,2,61,Thursday,50,1066,91
23,Weston Opitz,9,Animal Health Entomology,305,3,2,69,Thursday,50,1052,60
26,Weston Opitz,9,Animal Health Entomology,305,3,3,40,Tuesday,50,1052,60
29,Weston Opitz,9,Animal Health Entomology,305,3,3,80,Thursday,50,1052,60


,Professor,Workload,Course_Name,Course_Code,Credit,Session,Period,Day_Offered,Max_Enrollement,Classroom,Room Capacity
48,William Morrison III,6,Conventional and Molecular Methods for Evaluat...,885,3,1,11,Monday,20,1063,47
49,William Morrison III,6,Conventional and Molecular Methods for Evaluat...,885,3,1,12,Monday,20,1029,47
50,William Morrison III,6,Conventional and Molecular Methods for Evaluat...,885,3,1,13,Monday,20,1063,47
51,William Morrison III,6,Conventional and Molecular Methods for Evaluat...,885,3,1,51,Wednesday,20,1063,47
52,William Morrison III,6,Conventional and Molecular Methods for Evaluat...,885,3,1,52,Wednesday,20,1029,47
53,William Morrison III,6,Conventional and Molecular Methods for Evaluat...,885,3,1,53,Wednesday,20,1063,47
172,William Morrison III,6,Introduction to Insect Physiology,675,3,1,21,Tuesday,40,1029,47
173,William Morrison III,6,Introduction to Insect Physiology,675,3,1,22,Tuesday,40,1029,47
174,William Morrison III,6,Introduction to Insect Physiology,675,3,1,23,Tuesday,40,1029,47
175,William Morrison III,6,Introduction to Insect Physiology,675,3,1,61,Thursday,40,1029,47
